## 1 Полный перебор для 3х недель

In [73]:
import itertools
import numpy as np

# Матрицы вероятностей переходов P[action][state_from][state_to]
transition_probabilities = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# Матрицы вознаграждений R[action][state_from][state_to]
rewards = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

actions = [0, 1, 2]  # Доступные действия (индексы)
states = [0, 1, 2]   # Индексы состояний: 0 - «Отличный», 1 - «Хороший», 2 - «Удовлетворительный»
months = 3  # Количество месяцев (длительность стратегии)
state_names = ["Отличный", "Хороший", "Удовлетворительный"]  # Человекочитаемые имена состояний
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]  # Названия действий

def calculate_expected_reward(strategy, initial_state, verbose=False):
    """
    Вычисление ожидаемого вознаграждения для заданной стратегии действий.
    
    Параметры:
    strategy (list[int]): Стратегия действий (порядок действий на каждый месяц).
    initial_state (int): Начальное состояние системы.
    verbose (bool): Выводить ли промежуточные результаты.
    
    Возвращает:
    float: Ожидаемое вознаграждение для стратегии.
    """
    # Инициализация вероятностей состояний: на старте мы находимся в начальном состоянии
    state_probabilities = [0.0, 0.0, 0.0]
    state_probabilities[initial_state] = 1.0
    
    if verbose:
        print(f"\n=== Расчёт для стратегии {strategy} ===")
        print(f"Действия: {[action_names[a] for a in strategy]}")
        print(f"Начальное состояние: {state_names[initial_state]}")
        print(f"Начальное распределение вероятностей: {np.round(state_probabilities, 3)}")
    
    total_reward = 0.0  # Общее ожидаемое вознаграждение
    
    # Процесс для каждого действия в стратегии
    for month, action in enumerate(strategy):
        if verbose:
            print(f"\nМесяц {month+1}: Действие = {action} ({action_names[action]})")
            print(f"  Текущие вероятности состояний: {np.round(state_probabilities, 3)}")
        
        step_reward = 0.0  # Вознаграждение за текущий месяц
        new_state_probabilities = [0.0, 0.0, 0.0]  # Вероятности для нового состояния
        
        # Проходим по всем возможным переходам между состояниями
        for i in range(3):
            if state_probabilities[i] > 0:  # Оптимизация: показываем только значимые переходы
                if verbose:
                    print(f"  Из состояния {i} ({state_names[i]}) с вероятностью {state_probabilities[i]:.3f}:")
                
                for j in range(3):
                    transition_probability = transition_probabilities[action][i][j]
                    reward = rewards[action][i][j]
                    contribution = state_probabilities[i] * transition_probability * reward
                    
                    if contribution > 0 and verbose:
                        print(f"    → в состояние {j} ({state_names[j]}) с вер. {transition_probability:.3f}, "
                              f"награда {reward}, вклад: {contribution:.3f}")
                    
                    total_reward += contribution
                    step_reward += contribution
                    new_state_probabilities[j] += state_probabilities[i] * transition_probability
        
        # Обновляем вероятности состояний для следующего шага
        state_probabilities = new_state_probabilities
        
        if verbose:
            print(f"  Вознаграждение за месяц {month+1}: {step_reward:.3f}")
            print(f"  Новые вероятности состояний: {np.round(state_probabilities, 3)}")
            print(f"  Накопленное вознаграждение: {total_reward:.3f}")
    
    if verbose:
        print(f"\nИтоговое ожидаемое вознаграждение для стратегии {strategy}: {total_reward:.3f}")
    
    return total_reward

best_strategies = {0: None, 1: None, 2: None}  # Словарь для хранения лучших стратегий
best_rewards = {0: -1e9, 1: -1e9, 2: -1e9}  # Словарь для хранения лучших вознаграждений

# Поиск наилучшей стратегии для каждого начального состояния
for initial_state in range(3):
    best_strategy = None
    best_reward = -1e9  # Начальная очень низкая оценка
    strategies_evaluated = 0
    improvements = 0
    
    print(f"\n{'='*80}")
    print(f"Поиск лучшей стратегии для начального состояния «{state_names[initial_state]}»")
    print(f"{'='*80}")
    
    # Подсчет общего количества стратегий
    total_strategies = len(actions) ** months
    print(f"Всего возможных стратегий: {total_strategies}")
    
    # Генерация всех возможных стратегий на протяжении заданного количества месяцев
    for strategy in itertools.product(actions, repeat=months):
        strategies_evaluated += 1
        
        # Подробный вывод для первой и последней стратегии
        verbose = strategies_evaluated == 1 or strategies_evaluated == total_strategies
        
        if verbose:
            print(f"\nПроверка стратегии {strategies_evaluated}/{total_strategies}: {strategy}")
        
        expected_reward = calculate_expected_reward(strategy, initial_state, verbose=verbose)
        
        # Обновляем лучшую стратегию, если текущая дает большее вознаграждение
        if expected_reward > best_reward:
            improvement = expected_reward - best_reward if best_reward != -1e9 else expected_reward
            improvements += 1
            print(f"Улучшение #{improvements}: стратегия {strategy} = {[action_names[a] for a in strategy]}, "
                  f"вознаграждение: {expected_reward:.2f} (+{improvement:.2f})")
            best_strategy = strategy
            best_reward = expected_reward
            best_strategies[initial_state] = best_strategy
            best_rewards[initial_state] = best_reward
    
    # Выводим результаты для каждого начального состояния
    print(f"\n{'-'*80}")
    print(f"Для начального состояния «{state_names[initial_state]}»:")
    print(f"  Лучшая стратегия действий: {best_strategy} = {[action_names[a] for a in strategy]}")
    print(f"  Ожидаемое вознаграждение: {best_reward:.2f}")
    print(f"  Количество улучшений: {improvements} из {strategies_evaluated} оцененных стратегий")
    print(f"{'-'*80}\n")

print(f"\n{'='*40} Ответ {'='*40}")
for initial_state in range(3):
    print(f"Для начального состояния «{state_names[initial_state]}»:")
    print(f"  Лучшая стратегия действий: {best_strategies[initial_state]} = "
          f"{[action_names[a] for a in best_strategies[initial_state]]}")
    print(f"  Ожидаемое вознаграждение: {best_rewards[initial_state]:.2f}")
    print(f"{'-'*80}\n")


Поиск лучшей стратегии для начального состояния «Отличный»
Всего возможных стратегий: 27

Проверка стратегии 1/27: (0, 0, 0)

=== Расчёт для стратегии (0, 0, 0) ===
Действия: ['3% скидка', '3% скидка', '3% скидка']
Начальное состояние: Отличный
Начальное распределение вероятностей: [1. 0. 0.]

Месяц 1: Действие = 0 (3% скидка)
  Текущие вероятности состояний: [1. 0. 0.]
  Из состояния 0 (Отличный) с вероятностью 1.000:
    → в состояние 0 (Отличный) с вер. 0.300, награда 110, вклад: 33.000
    → в состояние 1 (Хороший) с вер. 0.500, награда 100, вклад: 50.000
    → в состояние 2 (Удовлетворительный) с вер. 0.200, награда 70, вклад: 14.000
  Вознаграждение за месяц 1: 97.000
  Новые вероятности состояний: [0.3 0.5 0.2]
  Накопленное вознаграждение: 97.000

Месяц 2: Действие = 0 (3% скидка)
  Текущие вероятности состояний: [0.3 0.5 0.2]
  Из состояния 0 (Отличный) с вероятностью 0.300:
    → в состояние 0 (Отличный) с вер. 0.300, награда 110, вклад: 9.900
    → в состояние 1 (Хороший) с

___
## 2 Полный перебор для бесконечного горизонта планирования

In [74]:
import itertools
import numpy as np

# Матрицы переходов и доходов для каждого действия
transition_matrix = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

reward_matrix = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

actions = [0, 1, 2]  # Индексы доступных действий
states = [0, 1, 2]   # Состояния: 0 - «Отличный», 1 - «Хороший», 2 - «Удовлетворительный»
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

def evaluate_policy(policy, verbose=True):
    """
    Оценка политики: вычисление среднего дохода и стационарного распределения состояний.
    
    Параметры:
    policy (list[int]): Стратегия действий для каждого состояния.
    verbose (bool): Выводить ли детальные промежуточные результаты.
    
    Возвращает:
    float: Средний доход от применения политики.
    numpy.ndarray: Стационарное распределение состояний.
    """
    # Инициализация матрицы переходов и вектора вознаграждений для данной политики
    transition_matrix_policy = np.zeros((3, 3))
    reward_vector_policy = np.zeros(3)
    
    if verbose:
        print(f"\n{'-'*80}")
        policy_str = ", ".join([f"s{i}→a{a}({action_names[a]})" for i, a in enumerate(policy)])
        print(f"Оценка политики: {policy} [{policy_str}]")
        print(f"{'-'*80}")
    
    # Для каждого состояния, вычисляем ожидаемое вознаграждение и вероятности перехода
    for state in states:
        action = policy[state]
        transition_matrix_policy[state, :] = transition_matrix[action][state]
        
        if verbose:
            print(f"\nСостояние {state} ({state_names[state]}), выбрано действие {action} ({action_names[action]}):")
            print(f"  Вероятности переходов P[{state},:] = {transition_matrix_policy[state, :]}")
        
        state_reward = 0
        for next_state in states:
            transition_prob = transition_matrix[action][state][next_state]
            reward = reward_matrix[action][state][next_state]
            contrib = transition_prob * reward
            state_reward += contrib
            
            if verbose:
                print(f"  Переход в {next_state} ({state_names[next_state]}): вероятность = {transition_prob:.3f}, "
                      f"награда = {reward}, вклад = {contrib:.3f}")
        
        reward_vector_policy[state] = state_reward
        
        if verbose:
            print(f"  Суммарное ожидаемое вознаграждение r[{state}] = {state_reward:.3f}")

    if verbose:
        print("\nМатрица переходов для политики:")
        for i in range(3):
            print(f"  {transition_matrix_policy[i, :]}")
        print("\nВектор вознаграждений для политики:")
        print(f"  {reward_vector_policy}")

    # Решение задачи с собственными значениями для нахождения стационарного распределения
    eigenvalues, eigenvectors = np.linalg.eig(transition_matrix_policy.T)
    
    if verbose:
        print("\nСобственные значения матрицы переходов:")
        for i, ev in enumerate(eigenvalues):
            print(f"  λ{i} = {ev:.6f} (|λ{i} - 1| = {abs(ev - 1.0):.6f})")
    
    stationary_state_idx = np.argmin(np.abs(eigenvalues - 1.0))
    
    if verbose:
        print(f"\nИндекс собственного значения, ближайшего к 1: {stationary_state_idx}")
        print(f"Соответствующий собственный вектор (ненормированный):")
        print(f"  {np.real(eigenvectors[:, stationary_state_idx])}")
    
    stationary_distribution = np.real(eigenvectors[:, stationary_state_idx])
    
    # Нормировка стационарного распределения
    stationary_distribution /= stationary_distribution.sum()

    # Рассчитываем средний доход
    average_reward = float(np.dot(stationary_distribution, reward_vector_policy))
    
    if verbose:
        print("\nСтационарное распределение состояний (нормированное):")
        for i, prob in enumerate(stationary_distribution):
            print(f"  μ({i}) = {prob:.6f} ({state_names[i]})")
        
        print("\nРасчет среднего дохода:")
        for i, (prob, reward) in enumerate(zip(stationary_distribution, reward_vector_policy)):
            print(f"  Состояние {i}: μ({i}) * r({i}) = {prob:.6f} * {reward:.3f} = {prob * reward:.6f}")
        
        print(f"\nСредний доход g = μ⋅r = {average_reward:.6f}")
    
    return average_reward, stationary_distribution

# Инициализация переменных для поиска лучшей политики
best_policy = None
best_gain = -1e9  # Начальное значение для максимального дохода
best_stationary_distribution = None

# Общее количество политик
total_policies = len(actions) ** len(states)
print(f"Всего возможных политик: {total_policies}")

# Перебор всех возможных политик и выбор лучшей
policy_count = 0
improvements = 0

for policy in itertools.product(actions, repeat=3):
    policy_count += 1
    policy = list(policy)  # Преобразование кортежа в список для удобства
    
    print(f"\n{'='*80}")
    print(f"Политика {policy_count}/{total_policies}: {policy} " + 
          f"[{', '.join([f's{i}→{a}({action_names[a]})' for i, a in enumerate(policy)])}]")
    
    # Подробный вывод только для первой, лучшей и последней политики
    verbose = (policy_count == 1) or (policy_count == total_policies)
    gain, stationary_distribution = evaluate_policy(policy, verbose=verbose)
    
    print(f"\nПолитика {policy}: средний доход g = {gain:.6f}")
    
    if gain > best_gain:
        improvements += 1
        improvement = gain - best_gain
        print(f"УЛУЧШЕНИЕ #{improvements}: +{improvement:.6f} (было {best_gain:.6f}, стало {gain:.6f})")
        best_gain = gain
        best_policy = policy
        best_stationary_distribution = stationary_distribution

print(f"\n\n\n{'='*40} Ответ {'='*40}")
print("Лучшая стационарная стратегия для состояний (0-Отличный, 1-Хороший, 2-Удовлетворительный):")
policy_str = ", ".join([f"s{i}→a{a}({action_names[a]})" for i, a in enumerate(best_policy)])
print(f"  {best_policy} [{policy_str}]")
print("\nСтационарное распределение состояний μ:")
for i, prob in enumerate(best_stationary_distribution):
    print(f"  μ({i}) = {prob:.6f} ({state_names[i]})")
print(f"\nСредний доход g при лучшей стратегии: {best_gain:.6f}")
print(f"Количество улучшений: {improvements} из {policy_count} проверенных политик")

print(f"\n\n\n{'='*40} Лучшая политика {'='*40}")
evaluate_policy(best_policy, verbose=True)

Всего возможных политик: 27

Политика 1/27: [0, 0, 0] [s0→0(3% скидка), s1→0(3% скидка), s2→0(3% скидка)]

--------------------------------------------------------------------------------
Оценка политики: [0, 0, 0] [s0→a0(3% скидка), s1→a0(3% скидка), s2→a0(3% скидка)]
--------------------------------------------------------------------------------

Состояние 0 (Отличный), выбрано действие 0 (3% скидка):
  Вероятности переходов P[0,:] = [0.3 0.5 0.2]
  Переход в 0 (Отличный): вероятность = 0.300, награда = 110, вклад = 33.000
  Переход в 1 (Хороший): вероятность = 0.500, награда = 100, вклад = 50.000
  Переход в 2 (Удовлетворительный): вероятность = 0.200, награда = 70, вклад = 14.000
  Суммарное ожидаемое вознаграждение r[0] = 97.000

Состояние 1 (Хороший), выбрано действие 0 (3% скидка):
  Вероятности переходов P[1,:] = [0.2 0.6 0.2]
  Переход в 0 (Отличный): вероятность = 0.200, награда = 100, вклад = 20.000
  Переход в 1 (Хороший): вероятность = 0.600, награда = 80, вклад = 48.000


(80.8641975308642, array([0.11111111, 0.38271605, 0.50617284]))

___
## 3 Метод итерации по стратегиям без дисконтирования

In [75]:
import numpy as np

# 1) Задаём P и R по условию задачи
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

num_states = 3
actions = [0, 1, 2]  # 0=скидка, 1=доставка, 2=ничего
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

def evaluate_policy(policy, verbose=True):
    """Policy evaluation (gain g и bias h)"""
    m = num_states
    # 1) Собираем P_pi и r_pi
    P_pi = np.zeros((m, m))
    r_pi = np.zeros(m)

    if verbose:
        print("\n" + "="*70)
        print(f"ОЦЕНКА ПОЛИТИКИ: {policy} ({[action_names[a] for a in policy]})")
        print("="*70)
        print("\n1) Собираем P_π и r_π на основе текущей политики:")
    
    for i in range(m):
        a = policy[i]
        if verbose:
            print(f"\n  Состояние {i} ({state_names[i]}) → действие {a} ({action_names[a]}):")
        
        # Заполнение строк матрицы переходов P_pi
        for j in range(m):
            P_pi[i, j] = P[a][i][j]
            r_pi[i] += P[a][i][j] * R[a][i][j]
            
            if verbose:
                print(f"    Переход в состояние {j} ({state_names[j]}): P[{a}][{i}][{j}] = {P[a][i][j]:.3f}, " + 
                      f"R[{a}][{i}][{j}] = {R[a][i][j]}, вклад в r_π[{i}] += {P[a][i][j] * R[a][i][j]:.3f}")
    
    if verbose:
        print("\n  Итоговая матрица переходов P_π:")
        for i in range(m):
            print(f"    {P_pi[i, :]}")
        print("\n  Итоговый вектор вознаграждений r_π:")
        print(f"    {r_pi}")

    # 2) Строим и решаем систему (m+1)x(m+1) на [h0..h_{m-1}, g]
    A = np.zeros((m + 1, m + 1))
    b = np.zeros(m + 1)
    
    if verbose:
        print("\n2) Строим систему уравнений (m+1)×(m+1) для нахождения [h₀, h₁, h₂, g]:")
        print("   (I - P_π)h + g·1 = r_π, с дополнительным условием h[m-1] = 0")
    
    for i in range(m):
        A[i, i] = 1.0
        A[i, :m] -= P_pi[i, :]
        A[i, m] = 1.0
        b[i] = r_pi[i]
        
        if verbose:
            eq_parts = []
            for j in range(m):
                if j == i:
                    coef = 1.0 - P_pi[i][j]
                else:
                    coef = -P_pi[i][j]
                
                if abs(coef) > 1e-6:  # Если коэффициент не слишком мал
                    sign = '+' if coef >= 0 and j > 0 else ''
                    eq_parts.append(f"{sign}{coef:.3f}·h[{j}]")
            
            eq_parts.append("+ g")
            eq = " ".join(eq_parts)
            print(f"    Уравнение {i+1}: {eq} = {r_pi[i]:.3f}")
    
    # фиксация h[m-1] = 0
    A[m, m - 1] = 1.0
    b[m] = 0.0
    
    if verbose:
        print(f"    Дополнительное условие: h[{m-1}] = 0")
        print("\n  Матрица системы A:")
        for i in range(m+1):
            print(f"    {A[i, :]}")
        print("\n  Вектор правых частей b:")
        print(f"    {b}")

    # Решаем систему
    x = np.linalg.solve(A, b)
    h = x[:m]
    g = x[m]
    
    if verbose:
        print("\n3) Решение системы уравнений:")
        print(f"  Вектор смещений h = {h}")
        print(f"  Средний доход g = {g:.6f}")
        
        # Проверка решения
        print("\n4) Проверка решения (должно быть приблизительно равно r_π):")
        for i in range(m):
            check_sum = 0
            for j in range(m):
                check_sum += P_pi[i, j] * h[j]
            check_val = h[i] - check_sum + g
            error = abs(check_val - r_pi[i])
            print(f"  Уравнение {i+1}: h[{i}] - ∑_j P_π[{i},{j}]·h[j] + g = {check_val:.6f}, r_π[{i}] = {r_pi[i]:.6f}, ошибка = {error:.6e}")

    return g, h

def improve_policy(policy, h, verbose=True):
    """Policy improvement"""
    m = num_states
    new_pol = policy.copy()
    
    if verbose:
        print(f"\n" + "="*40 + "Улучшение политики" + "="*40)
        print(f"\nТекущая политика: {policy} ({[action_names[a] for a in policy]})")
        print(f"Вектор смещений h = {h}")
    
    for i in range(m):
        if verbose:
            print(f"\nДля состояния {i} ({state_names[i]}) ищем оптимальное действие:")
        
        best_q, best_a = -1e9, None
        for a in actions:
            # считаем Q(i,a) = r(i,a) + sum_j P[a][i][j]·h[j]
            r_ia = 0.0
            bias_term = 0.0
            
            for j in range(m):
                r_ia += P[a][i][j] * R[a][i][j]
                bias_term += P[a][i][j] * h[j]
            
            q = r_ia + bias_term
            
            if verbose:
                print(f"  Действие {a} ({action_names[a]}):")
                # Детально показываем расчет r(i,a)
                print(f"    r({i},{a}) = ", end="")
                for j in range(m):
                    print(f"P[{a}][{i}][{j}]·R[{a}][{i}][{j}] = {P[a][i][j]:.3f}·{R[a][i][j]} = {P[a][i][j] * R[a][i][j]:.3f}", end="")
                    if j < m-1:
                        print(" + ", end="")
                print(f" = {r_ia:.3f}")
                
                # Детально показываем расчет bias-терма
                print(f"    Bias term = ", end="")
                for j in range(m):
                    print(f"P[{a}][{i}][{j}]·h[{j}] = {P[a][i][j]:.3f}·{h[j]:.3f} = {P[a][i][j] * h[j]:.3f}", end="")
                    if j < m-1:
                        print(" + ", end="")
                print(f" = {bias_term:.3f}")
                
                # Итоговый Q-value
                print(f"    Q({i},{a}) = {r_ia:.3f} + {bias_term:.3f} = {q:.3f}")
            
            if q > best_q:
                if verbose and best_a is not None:
                    print(f"    Лучше предыдущего действия {best_a} с Q = {best_q:.3f}, обновляем.")
                best_q, best_a = q, a
            elif verbose:
                print(f"    Хуже текущего лучшего действия {best_a} с Q = {best_q:.3f}, пропускаем.")
        
        new_pol[i] = best_a
        
        if verbose:
            action_changed = policy[i] != new_pol[i]
            print(f"  Лучшее действие для состояния {i}: {best_a} ({action_names[best_a]}), Q = {best_q:.3f}")
            if action_changed:
                print(f"  >>> Политика ИЗМЕНЕНА для состояния {i}: {policy[i]} → {new_pol[i]}")
            else:
                print(f"  >>> Политика НЕ ИЗМЕНЕНА для состояния {i}: остается {policy[i]}")
    
    if verbose:
        print("\nИтог улучшения политики:")
        print(f"  Было: {policy} ({[action_names[a] for a in policy]})")
        print(f"  Стало: {new_pol} ({[action_names[a] for a in new_pol]})")
        if policy == new_pol:
            print("  Политика НЕ ИЗМЕНИЛАСЬ - достигнута оптимальная политика.")
        else:
            print("  Политика ИЗМЕНИЛАСЬ - требуется продолжить итерации.")
            
    return new_pol

def policy_iteration():
    policy = [0] * num_states  # стартуем, например, всегда со "скидки"
    iteration = 0
    
    print("\nНачальная политика:")
    for i, a in enumerate(policy):
        print(f"  Состояние {i} ({state_names[i]}) → Действие {a} ({action_names[a]})")
    
    while True:
        print(f"\n{'*'*40} Итерация {iteration+1} {'*'*40}")
        
        print(f"\nТекущая политика:")
        for i, a in enumerate(policy):
            print(f"  {i} ({state_names[i]}) → {a} ({action_names[a]})")

        # Оценка этой политики
        g, h = evaluate_policy(policy)
        print(f"\nОценка текущей политики:")
        print(f"  Средний доход g = {g:.4f}")

        # Улучшаем политику
        new_pol = improve_policy(policy, h)

        # Если не изменилось — готово
        if new_pol == policy:
            print("\n" + "="*80)
            print("Политика не изменилась. Алгоритм завершён.")
            print("="*80)
            break

        policy = new_pol
        iteration += 1

    return policy, g, h

if __name__ == "__main__":
    opt_policy, opt_gain, opt_h = policy_iteration()
    print(f"\n\n\n{'='*40} Ответ {'='*40}")

    for i, a in enumerate(opt_policy):
        print(f"  Состояние {i} ({state_names[i]}) → Действие {a} ({action_names[a]})")
    print(f"\nОптимальный средний доход g* = {opt_gain:.4f}")
    print("="*80)


Начальная политика:
  Состояние 0 (Отличный) → Действие 0 (3% скидка)
  Состояние 1 (Хороший) → Действие 0 (3% скидка)
  Состояние 2 (Удовлетворительный) → Действие 0 (3% скидка)

**************************************** Итерация 1 ****************************************

Текущая политика:
  0 (Отличный) → 0 (3% скидка)
  1 (Хороший) → 0 (3% скидка)
  2 (Удовлетворительный) → 0 (3% скидка)

ОЦЕНКА ПОЛИТИКИ: [0, 0, 0] (['3% скидка', '3% скидка', '3% скидка'])

1) Собираем P_π и r_π на основе текущей политики:

  Состояние 0 (Отличный) → действие 0 (3% скидка):
    Переход в состояние 0 (Отличный): P[0][0][0] = 0.300, R[0][0][0] = 110, вклад в r_π[0] += 33.000
    Переход в состояние 1 (Хороший): P[0][0][1] = 0.500, R[0][0][1] = 100, вклад в r_π[0] += 50.000
    Переход в состояние 2 (Удовлетворительный): P[0][0][2] = 0.200, R[0][0][2] = 70, вклад в r_π[0] += 14.000

  Состояние 1 (Хороший) → действие 0 (3% скидка):
    Переход в состояние 0 (Отличный): P[0][1][0] = 0.200, R[0][1][0] =

___
## 4 Метод итерации по стратегии с дисконтированием

In [76]:
import numpy as np

# 1) Задаём P и R по условию
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

num_states = 3
actions = [0, 1, 2]  # 0=скидка, 1=доставка, 2=ничего
state_names = ["Отл.", "Хор.", "Уд."]
action_names = ["3% скидка", "доставка", "ничего"]

γ = 0.7  # коэффициент дисконтирования

def evaluate_policy_discount(policy, gamma=γ, verbose=True):
    """
    Решаем (I - γ Pπ) V = rπ
    возвращаем вектор V размера m.
    """
    m = num_states
    Pπ = np.zeros((m, m))
    rπ = np.zeros(m)
    
    if verbose:
        print("\n" + "="*40 + "ОЦЕНКА ПОЛИТИКИ" + "="*40)
        print(f"Действия политики: {[action_names[a] for a in policy]}")
        print("\n1) Создаём матрицу переходов Pπ и вектор вознаграждений rπ для текущей политики:")
    
    for i in range(m):
        a = policy[i]
        if verbose:
            print(f"\n  Состояние {i} ({state_names[i]}) → действие {a} ({action_names[a]}):")
        
        for j in range(m):
            Pπ[i, j] = P[a][i][j]
            rπ[i] += P[a][i][j] * R[a][i][j]
            
            if verbose:
                print(f"    Переход в {j} ({state_names[j]}): P[{a}][{i}][{j}] = {P[a][i][j]:.3f}, " +
                      f"R[{a}][{i}][{j}] = {R[a][i][j]}, вклад в rπ[{i}]: {P[a][i][j] * R[a][i][j]:.3f}")
    
    if verbose:
        print("\n  Итоговая матрица переходов Pπ:")
        for i in range(m):
            print(f"    {Pπ[i, :]}")
            
        print("\n  Итоговый вектор вознаграждений rπ:")
        print(f"    {rπ}")
        
        print("\n2) Формируем систему уравнений (I - γ·Pπ)·V = rπ:")
    
    # матрица I - γ·Pπ
    A = np.eye(m) - gamma * Pπ
    
    if verbose:
        print(f"\n  Матрица γ·Pπ (γ = {gamma}):")
        for i in range(m):
            print(f"    {gamma * Pπ[i, :]}")
            
        print("\n  Матрица системы A = I - γ·Pπ:")
        for i in range(m):
            print(f"    {A[i, :]}")
            
        print("\n  Система уравнений:")
        for i in range(m):
            eq_parts = []
            for j in range(m):
                if abs(A[i, j]) > 1e-10:
                    sign = "+" if A[i, j] > 0 and j > 0 else ""
                    eq_parts.append(f"{sign}{A[i, j]:.3f}·V[{j}]")
            
            eq = " ".join(eq_parts)
            print(f"    {eq} = {rπ[i]:.3f}")
    
    # Решаем систему
    V = np.linalg.solve(A, rπ)
    
    if verbose:
        print("\n3) Решение системы дает значения функции ценности V:")
        for i in range(m):
            print(f"    V[{i}] ({state_names[i]}) = {V[i]:.6f}")
            
        # Проверка решения
        print("\n4) Проверка решения (A·V должно быть ≈ rπ):")
        for i in range(m):
            check_val = 0
            for j in range(m):
                check_val += A[i, j] * V[j]
            error = abs(check_val - rπ[i])
            print(f"    Уравнение {i+1}: {check_val:.6f} ≈ {rπ[i]:.6f}, ошибка = {error:.6e}")
    
    return V

def improve_policy_discount(policy, V, gamma=γ, verbose=True):
    """
    Для каждого состояния i находим действие a, максимизирующее
    Q(i,a) = r(i,a) + γ ∑_j P[a][i][j]·V[j]
    """
    m = num_states
    new_pol = policy.copy()
    
    if verbose:
        print("\n" + "="*40 + "Улучшение политики" + "="*40)
        print(f"\nТекущая политика: {policy} ({[action_names[a] for a in policy]})")
        print(f"Текущие значения V: {np.round(V, 4)}")
    
    for i in range(m):
        if verbose:
            print(f"\nДля состояния {i} ({state_names[i]}) ищем оптимальное действие:")
        
        best_q, best_a = -float('inf'), None
        q_values = []
        
        for a in actions:
            # Вычисляем Q(i,a) = r(i,a) + γ·∑_j P[a][i][j]·V[j]
            # 1. Мгновенное вознаграждение r(i,a)
            r_ia = 0.0
            for j in range(m):
                r_ia += P[a][i][j] * R[a][i][j]
            
            # 2. Дисконтированное будущее вознаграждение
            future_val = 0.0
            for j in range(m):
                future_val += gamma * P[a][i][j] * V[j]
            
            # 3. Суммарное Q-значение
            q = r_ia + future_val
            q_values.append(q)
            
            if verbose:
                print(f"  Действие {a} ({action_names[a]}):")
                
                # Детально показываем вычисление r(i,a)
                print(f"    r({i},{a}) = ", end="")
                for j in range(m):
                    print(f"P[{a}][{i}][{j}]·R[{a}][{i}][{j}] = {P[a][i][j]:.3f}·{R[a][i][j]} = {P[a][i][j] * R[a][i][j]:.3f}", end="")
                    if j < m-1:
                        print(" + ", end="")
                print(f" = {r_ia:.3f}")
                
                # Детально показываем вычисление дисконтированного будущего
                print(f"    γ·∑_j P[{a}][{i}][j]·V[j] = ", end="")
                for j in range(m):
                    print(f"γ·P[{a}][{i}][{j}]·V[{j}] = {gamma:.1f}·{P[a][i][j]:.3f}·{V[j]:.3f} = {gamma * P[a][i][j] * V[j]:.3f}", end="")
                    if j < m-1:
                        print(" + ", end="")
                print(f" = {future_val:.3f}")
                
                # Итоговое Q-значение
                print(f"    Q({i},{a}) = {r_ia:.3f} + {future_val:.3f} = {q:.3f}")
            
            if q > best_q:
                if verbose and best_a is not None:
                    print(f"    Лучше предыдущего действия {best_a} с Q={best_q:.3f}, обновляем.")
                best_q, best_a = q, a
            elif verbose:
                print(f"    Хуже текущего лучшего действия {best_a} с Q={best_q:.3f}, пропускаем.")
        
        # Определяем лучшее действие для этого состояния
        new_pol[i] = best_a
        
        if verbose:
            action_changed = new_pol[i] != policy[i]
            print(f"  Лучшее действие для состояния {i}: {best_a} ({action_names[best_a]}), Q={best_q:.3f}")
            print(f"  Q-значения всех действий: {np.round(q_values, 3)}")
            
            if action_changed:
                print(f"  >>> Политика ИЗМЕНЕНА для состояния {i}: {policy[i]} → {new_pol[i]}")
            else:
                print(f"  >>> Политика НЕ ИЗМЕНЕНА для состояния {i}: остается {policy[i]}")
    
    if verbose:
        print("\nИтог улучшения политики:")
        print(f"  Было: {policy} ({[action_names[a] for a in policy]})")
        print(f"  Стало: {new_pol} ({[action_names[a] for a in new_pol]})")
        if policy == new_pol:
            print("  Политика НЕ ИЗМЕНИЛАСЬ - достигнута оптимальная политика.")
        else:
            print("  Политика ИЗМЕНИЛАСЬ - требуется продолжить итерации.")
    
    return new_pol

def policy_iteration_discount():
    # стартуем, например, всегда «3% скидка»
    policy = [0] * num_states
    it = 0
    
    print(f"\nКоэффициент дисконтирования γ = {γ}")
    print("\nНачальная политика:")
    for i, a in enumerate(policy):
        print(f"  Состояние {i} ({state_names[i]}) → Действие {a} ({action_names[a]})")
    
    while True:
        print(f"\n{'*'*40 } Итерация {it+1} {'*'*40}")
        
        print(f"\nТекущая политика:")
        for i, a in enumerate(policy):
            print(f"  {i} ({state_names[i]}) → {a} ({action_names[a]})")
        
        # оценка
        V = evaluate_policy_discount(policy)
        print(f"\nРезультат оценки политики:")
        print(f"  V = {np.round(V, 3)}")
        
        # улучшение
        new_pol = improve_policy_discount(policy, V)
        
        # Проверка на сходимость
        if new_pol == policy:
            print("\n" + "="*80)
            print("Политика не изменилась. Алгоритм завершён.")
            print("="*80)
            break
        
        # Информация об изменении политики
        print("\nИзменения в политике:")
        for i in range(num_states):
            if new_pol[i] != policy[i]:
                print(f"  Состояние {i} ({state_names[i]}): {policy[i]} ({action_names[policy[i]]}) → {new_pol[i]} ({action_names[new_pol[i]]})")
        
        policy = new_pol
        it += 1
    
    return policy, V

if __name__ == "__main__":
    opt_pol, opt_V = policy_iteration_discount()
    
    print("\n" + "="*40 + " Ответ " + "="*40)
    
    print("\nОптимальная политика:")
    for i, a in enumerate(opt_pol):
        print(f"  Состояние {i} ({state_names[i]}) → Действие {a} ({action_names[a]})")
    
    print("\nОптимальная функция ценности:")
    for i, v in enumerate(opt_V):
        print(f"  V*[{i}] ({state_names[i]}) = {v:.6f}")
    
    print("\nСтоимость состояний V* в округлённом виде:")
    print(f"  {np.round(opt_V, 3)}")
    


Коэффициент дисконтирования γ = 0.7

Начальная политика:
  Состояние 0 (Отл.) → Действие 0 (3% скидка)
  Состояние 1 (Хор.) → Действие 0 (3% скидка)
  Состояние 2 (Уд.) → Действие 0 (3% скидка)

**************************************** Итерация 1 ****************************************

Текущая политика:
  0 (Отл.) → 0 (3% скидка)
  1 (Хор.) → 0 (3% скидка)
  2 (Уд.) → 0 (3% скидка)

========================================ОЦЕНКА ПОЛИТИКИ========================================
Действия политики: ['3% скидка', '3% скидка', '3% скидка']

1) Создаём матрицу переходов Pπ и вектор вознаграждений rπ для текущей политики:

  Состояние 0 (Отл.) → действие 0 (3% скидка):
    Переход в 0 (Отл.): P[0][0][0] = 0.300, R[0][0][0] = 110, вклад в rπ[0]: 33.000
    Переход в 1 (Хор.): P[0][0][1] = 0.500, R[0][0][1] = 100, вклад в rπ[0]: 50.000
    Переход в 2 (Уд.): P[0][0][2] = 0.200, R[0][0][2] = 70, вклад в rπ[0]: 14.000

  Состояние 1 (Хор.) → действие 0 (3% скидка):
    Переход в 0 (Отл.): P[0]

___ 
## 5 решение методами линейного программирования (Без дисконтирования)

In [77]:
import numpy as np
from scipy.optimize import linprog


# 1. Определяем матрицы вероятностей P и вознаграждений R по условию задачи

# P[a][s][s'] - вероятность перехода из состояния s в s' при действии a
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# R[a][s][s'] - вознаграждение за переход из состояния s в s' при действии a
R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

# Состояния и действия
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

# Параметры задачи
num_states = 3  # Количество состояний
num_actions = 3  # Количество действий
num_variables = num_states * num_actions  # Общее количество переменных (9)


# 2. Формируем вектор вознаграждений r для каждого состояния и действия
print("\n1. ФОРМИРОВАНИЕ ВЕКТОРА ВОЗНАГРАЖДЕНИЙ")
print("-"*60)

reward_vector = np.zeros(num_variables)
print("\nРасчет ожидаемых моментальных вознаграждений r(s,a):")

for state in range(num_states):
    print(f"\nСостояние {state} ({state_names[state]}):")
    
    for action in range(num_actions):
        idx = state * num_actions + action
        # Вычисляем детальное ожидаемое вознаграждение
        reward = 0
        print(f"  Действие {action} ({action_names[action]}):")
        
        for next_state in range(num_states):
            contribution = P[action][state][next_state] * R[action][state][next_state]
            reward += contribution
            print(f"    Переход в {next_state} ({state_names[next_state]}): " + 
                  f"P={P[action][state][next_state]:.3f}, R={R[action][state][next_state]}, " + 
                  f"вклад = {contribution:.3f}")
        
        reward_vector[idx] = reward
        print(f"  Итоговое r({state},{action}) = {reward:.3f} (индекс в векторе reward_vector: {idx})")

print("\nИтоговый вектор вознаграждений reward_vector:")
for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    print(f"  reward_vector[{i}] = r({state},{action}) = {reward_vector[i]:.3f}")

# 3. Формируем матрицу A_eq и вектор b_eq для решения задачи линейного программирования
print("\n2. ФОРМИРОВАНИЕ СИСТЕМЫ ОГРАНИЧЕНИЙ")
print("-"*60)

# A_eq и b_eq для условия балансировки потоков и нормировки
A_eq = np.zeros((num_states + 1, num_variables))
b_eq = np.zeros(num_states + 1)

print("\nФормирование условий балансировки потоков:")
# а) Баланс потоков: для каждого состояния j, учитываем все действия a
for next_state in range(num_states):
    print(f"\nДля состояния {next_state} ({state_names[next_state]}):")
    equation_terms = []
    
    for state in range(num_states):
        for action in range(num_actions):
            idx = state * num_actions + action
            coef = 0
            
            # Если это состояние j, добавляем +1*x(j,a)
            if state == next_state:
                A_eq[next_state, idx] += 1.0
                coef += 1.0
                equation_terms.append(f"+1.0·x({state},{action})")
            
            # Вычитаем вероятность перехода в j из любого состояния
            transition_prob = P[action][state][next_state]
            A_eq[next_state, idx] -= transition_prob
            coef -= transition_prob
            
            if abs(coef) > 1e-10:  # Показываем только значимые коэффициенты
                print(f"  x({state},{action}) с коэффициентом {coef:.3f}")
                if state == next_state:
                    print(f"    = 1.0 (потому что это текущее состояние) - {transition_prob:.3f} (P[{action}][{state}][{next_state}])")
                else:
                    print(f"    = 0.0 (не текущее состояние) - {transition_prob:.3f} (P[{action}][{state}][{next_state}])")

    print(f"  Полное уравнение баланса (в матричном виде): ∑ коэффициентов · x(s,a) = 0")

print("\nУсловие нормировки:")
# б) Нормировка: сумма всех переменных x_{s,a} равна 1
A_eq[num_states, :] = 1.0
b_eq[num_states] = 1.0

print("  Сумма всех x(s,a) = 1.0:")
for state in range(num_states):
    for action in range(num_actions):
        print(f"    + x({state},{action})")

print("\nИтоговая матрица коэффициентов A_eq:")
for i in range(A_eq.shape[0]):
    if i < num_states:
        row_desc = f"Уравнение баланса для состояния {i} ({state_names[i]})"
    else:
        row_desc = "Условие нормировки"
    print(f"  Строка {i} ({row_desc}):")
    
    # Выводим значения по частям для лучшей читаемости
    for j in range(num_variables):
        state_j = j // num_actions
        action_j = j % num_actions
        if abs(A_eq[i, j]) > 1e-10:  # Только значимые коэффициенты
            print(f"    A_eq[{i},{j}] (для x({state_j},{action_j})) = {A_eq[i, j]:.4f}")

print("\nИтоговый вектор правых частей b_eq:")
for i in range(len(b_eq)):
    if i < num_states:
        row_desc = f"Уравнение баланса для состояния {i} ({state_names[i]})"
    else:
        row_desc = "Условие нормировки"
    print(f"  b_eq[{i}] ({row_desc}) = {b_eq[i]}")

print("\nСистема уравнений в развернутом виде:")
for i in range(num_states):
    equation_parts = []
    for j in range(num_states * num_actions):
        if abs(A_eq[i, j]) > 1e-10:
            state_j = j // num_actions
            action_j = j % num_actions
            sign = "+" if A_eq[i, j] > 0 and equation_parts else ""
            equation_parts.append(f"{sign}{A_eq[i, j]:.4f}·y({state_j},{action_j})")
    
    equation = " ".join(equation_parts)
    print(f"  {equation} = {b_eq[i]}")


# 4. Решаем задачу линейного программирования для максимизации r^T * x
print("\n3. РЕШЕНИЕ ЗАДАЧИ ЛИНЕЙНОГО ПРОГРАММИРОВАНИЯ")
print("-"*60)

print("\nЗадача: максимизировать r^T·x при ограничениях A_eq·x = b_eq, x ≥ 0")
print("  где x - вектор переменных x(s,a), r - вектор моментальных вознаграждений")
print("  При этом для решения задачи максимизации через linprog (который минимизирует),")
print("  мы минимизируем -r^T·x")

print("\nВектор коэффициентов целевой функции (-r):")
for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    print(f"  c[{i}] для x({state},{action}) = {-reward_vector[i]:.4f}")

# Задача сводится к минимизации -r^T * x
result = linprog(
    c=-reward_vector,  # Минмизируем -r^T * x
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=[(0, None)] * num_variables,
    method='highs'  # Используем метод 'highs', возможен также 'revised simplex'
)

# Проверка успешности решения задачи
if not result.success:
    print(f"\nОшибка в решении LP задачи: {result.message}")
else:
    print(f"\nЗадача успешно решена за {result.nit} итераций.")
    print(f"Статус: {result.message}")

# Оптимальное решение
optimal_x = result.x
optimal_reward = reward_vector.dot(optimal_x)

print("\nОптимальные значения переменных x(s,a):")
for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    print(f"  x({state},{action}) = {optimal_x[i]:.6f}")

print(f"\nОптимальное значение целевой функции (средний доход): {optimal_reward:.6f}")

# 5. Восстанавливаем политику для каждого состояния
print("\n4. ВОССТАНОВЛЕНИЕ ОПТИМАЛЬНОЙ ПОЛИТИКИ")
print("-"*60)

print("\nДля каждого состояния выбираем действие с наибольшим значением x(s,a):")
optimal_policy = []

for state in range(num_states):
    print(f"\nСостояние {state} ({state_names[state]}):")
    values = [optimal_x[state * num_actions + action] for action in range(num_actions)]
    
    print("  Значения x(s,a) для разных действий:")
    for action in range(num_actions):
        print(f"    x({state},{action}) = {values[action]:.6f}")
    
    best_action = int(np.argmax(values))  # Действие с максимальной вероятностью
    optimal_policy.append(best_action)
    
    print(f"  Максимальное значение: x({state},{best_action}) = {values[best_action]:.6f}")
    print(f"  Выбранное оптимальное действие: {best_action} ({action_names[best_action]})")

# 6. Выводим итоговые результаты
print("\n5. ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("-"*60)

print(f"\nОптимальное среднее вознаграждение g* = {optimal_reward:.4f}")
print("\nОптимальная стационарная детерминированная политика:")
for state in range(num_states):
    print(f"  Состояние {state} ({state_names[state]}) → Действие {optimal_policy[state]} ({action_names[optimal_policy[state]]})")

print("\nСтационарное распределение по состояниям:")
for state in range(num_states):
    state_sum = sum(optimal_x[state * num_actions + action] for action in range(num_actions))
    print(f"  Состояние {state} ({state_names[state]}): {state_sum:.6f}")


1. ФОРМИРОВАНИЕ ВЕКТОРА ВОЗНАГРАЖДЕНИЙ
------------------------------------------------------------

Расчет ожидаемых моментальных вознаграждений r(s,a):

Состояние 0 (Отличный):
  Действие 0 (3% скидка):
    Переход в 0 (Отличный): P=0.300, R=110, вклад = 33.000
    Переход в 1 (Хороший): P=0.500, R=100, вклад = 50.000
    Переход в 2 (Удовлетворительный): P=0.200, R=70, вклад = 14.000
  Итоговое r(0,0) = 97.000 (индекс в векторе reward_vector: 0)
  Действие 1 (Бесплатная доставка):
    Переход в 0 (Отличный): P=0.200, R=120, вклад = 24.000
    Переход в 1 (Хороший): P=0.700, R=100, вклад = 70.000
    Переход в 2 (Удовлетворительный): P=0.100, R=70, вклад = 7.000
  Итоговое r(0,1) = 101.000 (индекс в векторе reward_vector: 1)
  Действие 2 (Ничего):
    Переход в 0 (Отличный): P=0.300, R=110, вклад = 33.000
    Переход в 1 (Хороший): P=0.400, R=80, вклад = 32.000
    Переход в 2 (Удовлетворительный): P=0.300, R=50, вклад = 15.000
  Итоговое r(0,2) = 80.000 (индекс в векторе reward_vec

___
## 5 решение методами линейного программирования (С дисконтированием)

In [ ]:
import numpy as np
from scipy.optimize import linprog


# P[a][s][s'] - вероятность перехода из состояния s в состояние s' при действии a
transition_probabilities = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# R[a][s][s'] - вознаграждение за переход из состояния s в s' при действии a
rewards = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

# Параметры задачи
num_states = 3  # Количество состояний
num_actions = 3  # Количество действий
discount_factor = 0.7  # Дисконт-фактор
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

# Начальное распределение: стартуем из состояния "Отличный"
initial_distribution = np.array([1.0, 0.0, 0.0])


# 1. Формируем вектор вознаграждений r размером num_states * num_actions
print("\n1. ФОРМИРОВАНИЕ ВЕКТОРА ВОЗНАГРАЖДЕНИЙ")
print("-"*60)

reward_vector = np.zeros(num_states * num_actions)
print("\nРасчет ожидаемых моментальных вознаграждений r(s,a):")

for state in range(num_states):
    print(f"\nСостояние {state} ({state_names[state]}):")
    
    for action in range(num_actions):
        idx = state * num_actions + action
        # Вычисляем детальное ожидаемое вознаграждение
        reward = 0
        print(f"  Действие {action} ({action_names[action]}):")
        
        for next_state in range(num_states):
            contribution = transition_probabilities[action][state][next_state] * rewards[action][state][next_state]
            reward += contribution
            print(f"    Переход в {next_state} ({state_names[next_state]}): " + 
                  f"P={transition_probabilities[action][state][next_state]:.3f}, R={rewards[action][state][next_state]}, " + 
                  f"вклад = {contribution:.3f}")
        
        reward_vector[idx] = reward
        print(f"  Итоговое r({state},{action}) = {reward:.3f} (индекс в векторе reward_vector: {idx})")

print("\nИтоговый вектор вознаграждений reward_vector:")
for i in range(num_states * num_actions):
    state = i // num_actions
    action = i % num_actions
    print(f"  reward_vector[{i}] = r({state},{action}) = {reward_vector[i]:.3f}")

# 3. Формируем матрицу равенств A_eq и вектор b_eq для линейного программирования
print("\n2. ФОРМИРОВАНИЕ СИСТЕМЫ УРАВНЕНИЙ ДЛЯ ДИСКОНТИРОВАННОГО СЛУЧАЯ")
print("-"*60)

# A_eq - матрица коэффициентов, b_eq - вектор правых частей
A_eq = np.zeros((num_states, num_states * num_actions))
b_eq = initial_distribution.copy()

print("\nФормирование системы уравнений для дисконтированного MDP:")
print(f"  Для каждого состояния j: ∑_a y(j,a) - γ·∑_s,a P[a][s][j]·y(s,a) = d₀[j]")
print(f"  где γ = {discount_factor}, d₀ = {initial_distribution}")

# Формируем систему уравнений для каждого состояния
for next_state in range(num_states):
    print(f"\nДля состояния {next_state} ({state_names[next_state]}):")
    print(f"  d₀[{next_state}] = {initial_distribution[next_state]}")
    
    for state in range(num_states):
        for action in range(num_actions):
            idx = state * num_actions + action
            
            # Коэффициент при y(s,a) в уравнении для состояния next_state
            coef = 0
            
            # Добавляем +1 для y(j,a) в левой части, если state == next_state
            if state == next_state:
                A_eq[next_state, idx] += 1.0
                coef += 1.0
                print(f"  Для y({state},{action}): +1.0 (т.к. это текущее состояние {next_state})")
            
            # Вычитаем γ·P[a][s][j] для всех y(s,a)
            transition_prob = transition_probabilities[action][state][next_state]
            discounted_prob = discount_factor * transition_prob
            A_eq[next_state, idx] -= discounted_prob
            coef -= discounted_prob
            
            if abs(discounted_prob) > 1e-10:
                print(f"  Для y({state},{action}): -{discount_factor:.1f}·{transition_prob:.3f} = -{discounted_prob:.3f} " +
                      f"(переход в состояние {next_state})")
            
            if abs(coef) > 1e-10:
                print(f"  Итоговый коэффициент для y({state},{action}): {coef:.4f}")

print("\nИтоговая матрица коэффициентов A_eq:")
for i in range(A_eq.shape[0]):
    print(f"  Строка {i} (уравнение для состояния {i} - {state_names[i]}):")
    
    # Выводим значения по группам для лучшей читаемости
    for state in range(num_states):
        print(f"    Для переменных состояния {state} ({state_names[state]}):", end=" ")
        for action in range(num_actions):
            idx = state * num_actions + action
            print(f"{A_eq[i, idx]:+.4f}", end=" ")
        print()

print("\nИтоговый вектор правых частей b_eq:")
for i in range(len(b_eq)):
    print(f"  b_eq[{i}] (для состояния {i} - {state_names[i]}) = {b_eq[i]}")

print("\nСистема уравнений в развернутом виде:")
for i in range(num_states):
    equation_parts = []
    for j in range(num_states * num_actions):
        if abs(A_eq[i, j]) > 1e-10:
            state_j = j // num_actions
            action_j = j % num_actions
            sign = "+" if A_eq[i, j] > 0 and equation_parts else ""
            equation_parts.append(f"{sign}{A_eq[i, j]:.4f}·y({state_j},{action_j})")
    
    equation = " ".join(equation_parts)
    print(f"  {equation} = {b_eq[i]}")

# 4. Решаем задачу линейного программирования (LP): maximize r^T * y <=> minimize -r^T * y
print("\n3. РЕШЕНИЕ ЗАДАЧИ ЛИНЕЙНОГО ПРОГРАММИРОВАНИЯ")
print("-"*60)

print("\nЗадача: максимизировать r^T·y при ограничениях A_eq·y = b_eq, y ≥ 0")
print("  где y - вектор переменных y(s,a), r - вектор моментальных вознаграждений")
print("  При решении через linprog (который минимизирует) используем -r^T·y")

print("\nВектор коэффициентов целевой функции (-r):")
for i in range(num_states * num_actions):
    state = i // num_actions
    action = i % num_actions
    print(f"  c[{i}] для y({state},{action}) = {-reward_vector[i]:.4f}")

# Задача сводится к минимизации -r^T * y
result = linprog(
    c=-reward_vector,  # Минимизируем -r^T * y
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=[(0, None)] * (num_states * num_actions),
    method='highs'  # Используем метод 'highs', можно также использовать 'revised simplex'
)

# Проверка успешности решения задачи
if not result.success:
    print(f"\nОшибка в решении LP задачи: {result.message}")
else:
    print(f"\nЗадача успешно решена за {result.nit} итераций.")
    print(f"Статус: {result.message}")

# Оптимальные значения переменных y
optimal_y = result.x
optimal_value = reward_vector.dot(optimal_y)  # Оптимальный дисконтированный доход

print("\nОптимальные значения переменных y(s,a):")
for i in range(num_states * num_actions):
    state = i // num_actions
    action = i % num_actions
    print(f"  y({state},{action}) = {optimal_y[i]:.6f}")

print(f"\nОптимальное значение целевой функции (дисконтированный доход): {optimal_value:.6f}")

# 5. Восстанавливаем стратегию (политику)
print("\n4. ВОССТАНОВЛЕНИЕ ОПТИМАЛЬНОЙ ПОЛИТИКИ")
print("-"*60)

print("\nДля каждого состояния выбираем действие с наибольшим значением y(s,a):")
optimal_policy = []

for state in range(num_states):
    print(f"\nСостояние {state} ({state_names[state]}):")
    action_values = [optimal_y[state * num_actions + action] for action in range(num_actions)]
    
    print("  Значения y(s,a) для разных действий:")
    for action in range(num_actions):
        print(f"    y({state},{action}) = {action_values[action]:.6f}")
    
    optimal_action = int(np.argmax(action_values))  # Действие с максимальной переменной
    optimal_policy.append(optimal_action)
    
    print(f"  Максимальное значение: y({state},{optimal_action}) = {action_values[optimal_action]:.6f}")
    print(f"  Выбранное оптимальное действие: {optimal_action} ({action_names[optimal_action]})")

# 6. Проверка и интерпретация результатов
print("\n5. ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("-"*60)

print(f"\nОптимальный дисконтированный доход V* = {optimal_value:.4f}")
print("\nОптимальная детерминированная политика:")
for state in range(num_states):
    print(f"  Состояние {state} ({state_names[state]}) → Действие {optimal_policy[state]} ({action_names[optimal_policy[state]]})")



1. ФОРМИРОВАНИЕ ВЕКТОРА ВОЗНАГРАЖДЕНИЙ
------------------------------------------------------------

Расчет ожидаемых моментальных вознаграждений r(s,a):

Состояние 0 (Отличный):
  Действие 0 (3% скидка):
    Переход в 0 (Отличный): P=0.300, R=110, вклад = 33.000
    Переход в 1 (Хороший): P=0.500, R=100, вклад = 50.000
    Переход в 2 (Удовлетворительный): P=0.200, R=70, вклад = 14.000
  Итоговое r(0,0) = 97.000 (индекс в векторе reward_vector: 0)
  Действие 1 (Бесплатная доставка):
    Переход в 0 (Отличный): P=0.200, R=120, вклад = 24.000
    Переход в 1 (Хороший): P=0.700, R=100, вклад = 70.000
    Переход в 2 (Удовлетворительный): P=0.100, R=70, вклад = 7.000
  Итоговое r(0,1) = 101.000 (индекс в векторе reward_vector: 1)
  Действие 2 (Ничего):
    Переход в 0 (Отличный): P=0.300, R=110, вклад = 33.000
    Переход в 1 (Хороший): P=0.400, R=80, вклад = 32.000
    Переход в 2 (Удовлетворительный): P=0.300, R=50, вклад = 15.000
  Итоговое r(0,2) = 80.000 (индекс в векторе reward_vec

___
## 6


In [79]:
import numpy as np
from scipy.optimize import linprog


# 1. Определяем матрицы вероятностей P и вознаграждений R по условию задачи

# P[a][s][s'] - вероятность перехода из состояния s в s' при действии a
P = {
    0: [[0.3, 0.5, 0.2],
        [0.2, 0.6, 0.2],
        [0.1, 0.2, 0.7]],
    1: [[0.2, 0.7, 0.1],
        [0.1, 0.4, 0.5],
        [0.1, 0.2, 0.7]],
    2: [[0.3, 0.4, 0.3],
        [0.2, 0.6, 0.2],
        [0.1, 0.3, 0.6]],
}

# R[a][s][s'] - вознаграждение за переход из состояния s в s' при действии a
R = {
    0: [[110, 100, 70],
        [100,  80, 50],
        [ 80,  60, 40]],
    1: [[120, 100, 70],
        [110, 100, 90],
        [100,  70, 60]],
    2: [[110,  80, 50],
        [100,  60, 40],
        [ 80,  70, 60]],
}

# Состояния и действия
state_names = ["Отличный", "Хороший", "Удовлетворительный"]
action_names = ["3% скидка", "Бесплатная доставка", "Ничего"]

# Параметры задачи
num_states = 3  # Количество состояний
num_actions = 3  # Количество действий
num_variables = num_states * num_actions  # Общее количество переменных (9)


# 2. Формируем вектор вознаграждений r для каждого состояния и действия
#print("\n1. ФОРМИРОВАНИЕ ВЕКТОРА ВОЗНАГРАЖДЕНИЙ")
#print("-"*60)

reward_vector = np.zeros(num_variables)
#print("\nРасчет ожидаемых моментальных вознаграждений r(s,a):")

for state in range(num_states):
    #print(f"\nСостояние {state} ({state_names[state]}):")
    
    for action in range(num_actions):
        idx = state * num_actions + action
        # Вычисляем детальное ожидаемое вознаграждение
        reward = 0
        #print(f"  Действие {action} ({action_names[action]}):")
        
        for next_state in range(num_states):
            contribution = P[action][state][next_state] * R[action][state][next_state]
            reward += contribution
            #print(f"    Переход в {next_state} ({state_names[next_state]}): " + 
            #      f"P={P[action][state][next_state]:.3f}, R={R[action][state][next_state]}, " + 
            #     f"вклад = {contribution:.3f}")
        
        reward_vector[idx] = reward
        #print(f"  Итоговое r({state},{action}) = {reward:.3f} (индекс в векторе reward_vector: {idx})")

#print("\nИтоговый вектор вознаграждений reward_vector:")
for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    #print(f"  reward_vector[{i}] = r({state},{action}) = {reward_vector[i]:.3f}")

# 3. Формируем матрицу A_eq и вектор b_eq для решения задачи линейного программирования
#print("\n2. ФОРМИРОВАНИЕ СИСТЕМЫ ОГРАНИЧЕНИЙ")
#print("-"*60)

# A_eq и b_eq для условия балансировки потоков и нормировки
A_eq = np.zeros((num_states + 1, num_variables))
b_eq = np.zeros(num_states + 1)

#print("\nФормирование условий балансировки потоков:")
# а) Баланс потоков: для каждого состояния j, учитываем все действия a
for next_state in range(num_states):
    #print(f"\nДля состояния {next_state} ({state_names[next_state]}):")
    equation_terms = []
    
    for state in range(num_states):
        for action in range(num_actions):
            idx = state * num_actions + action
            coef = 0
            
            # Если это состояние j, добавляем +1*x(j,a)
            if state == next_state:
                A_eq[next_state, idx] += 1.0
                coef += 1.0
                equation_terms.append(f"+1.0·x({state},{action})")
            
            # Вычитаем вероятность перехода в j из любого состояния
            transition_prob = P[action][state][next_state]
            A_eq[next_state, idx] -= transition_prob
            coef -= transition_prob
            
            if abs(coef) > 1e-10:  # Показываем только значимые коэффициенты
                #print(f"  x({state},{action}) с коэффициентом {coef:.3f}")
                if state == next_state:
                    #print(f"    = 1.0 (потому что это текущее состояние) - {transition_prob:.3f} (P[{action}][{state}][{next_state}])")
                    pass
                else:
                    pass
                    #print(f"    = 0.0 (не текущее состояние) - {transition_prob:.3f} (P[{action}][{state}][{next_state}])")

    #print(f"  Полное уравнение баланса (в матричном виде): ∑ коэффициентов · x(s,a) = 0")

#print("\nУсловие нормировки:")
# б) Нормировка: сумма всех переменных x_{s,a} равна 1
A_eq[num_states, :] = 1.0
b_eq[num_states] = 1.0

#print("  Сумма всех x(s,a) = 1.0:")
for state in range(num_states):
    for action in range(num_actions):
        #print(f"    + x({state},{action})")
        pass
#print("\nИтоговая матрица коэффициентов A_eq:")
for i in range(A_eq.shape[0]):
    if i < num_states:
        row_desc = f"Уравнение баланса для состояния {i} ({state_names[i]})"
    else:
        row_desc = "Условие нормировки"
    #print(f"  Строка {i} ({row_desc}):")
    
    # Выводим значения по частям для лучшей читаемости
    for j in range(num_variables):
        state_j = j // num_actions
        action_j = j % num_actions
        #if abs(A_eq[i, j]) > 1e-10:  # Только значимые коэффициенты
            #print(f"    A_eq[{i},{j}] (для x({state_j},{action_j})) = {A_eq[i, j]:.4f}")

print("\nИтоговый вектор правых частей b_eq:")
for i in range(len(b_eq)):
    if i < num_states:
        row_desc = f"Уравнение баланса для состояния {i} ({state_names[i]})"
    else:
        row_desc = "Условие нормировки"
    print(f"  b_eq[{i}] ({row_desc}) = {b_eq[i]}")

print("\nСистема уравнений в развернутом виде:")
for i in range(num_states):
    equation_parts = []
    for j in range(num_states * num_actions):
        if abs(A_eq[i, j]) > 1e-10:
            state_j = j // num_actions
            action_j = j % num_actions
            sign = "+" if A_eq[i, j] > 0 and equation_parts else ""
            equation_parts.append(f"{sign}{A_eq[i, j]:.4f}·y({state_j},{action_j})")
    
    equation = " ".join(equation_parts)
    print(f"  {equation} = {b_eq[i]}")


# 4. Решаем задачу линейного программирования для максимизации r^T * x
print("\n3. РЕШЕНИЕ ЗАДАЧИ ЛИНЕЙНОГО ПРОГРАММИРОВАНИЯ")
print("-"*60)

print("\nЗадача: максимизировать r^T·x при ограничениях A_eq·x = b_eq, x ≥ 0")
print("  где x - вектор переменных x(s,a), r - вектор моментальных вознаграждений")
print("  При этом для решения задачи максимизации через linprog (который минимизирует),")
print("  мы минимизируем -r^T·x")

print("\nВектор коэффициентов целевой функции (-r):")
for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    print(f"  c[{i}] для x({state},{action}) = {-reward_vector[i]:.4f}")

# Задача сводится к минимизации -r^T * x
result = linprog(
    c=-reward_vector,  # Минмизируем -r^T * x
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=[(0, None)] * num_variables,
    method='highs'  # Используем метод 'highs', возможен также 'revised simplex'
)

# Проверка успешности решения задачи
if not result.success:
    print(f"\nОшибка в решении LP задачи: {result.message}")
else:
    #print(f"\nЗадача успешно решена за {result.nit} итераций.")
    #print(f"Статус: {result.message}")
    pass

# Оптимальное решение
optimal_x = result.x
optimal_reward = reward_vector.dot(optimal_x)

#print("\nОптимальные значения переменных x(s,a):")
for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    print(f"  x({state},{action}) = {optimal_x[i]:.6f}")

#print(f"\nОптимальное значение целевой функции (средний доход): {optimal_reward:.6f}")

# 5. Восстанавливаем политику для каждого состояния
#print("\n4. ВОССТАНОВЛЕНИЕ ОПТИМАЛЬНОЙ ПОЛИТИКИ")
#print("-"*60)

#print("\nДля каждого состояния выбираем действие с наибольшим значением x(s,a):")
optimal_policy = []

for state in range(num_states):
    #print(f"\nСостояние {state} ({state_names[state]}):")
    values = [optimal_x[state * num_actions + action] for action in range(num_actions)]
    
    #print("  Значения x(s,a) для разных действий:")
    for action in range(num_actions):
        #print(f"    x({state},{action}) = {values[action]:.6f}")
        pass
    best_action = int(np.argmax(values))  # Действие с максимальной вероятностью
    optimal_policy.append(best_action)
    
    #print(f"  Максимальное значение: x({state},{best_action}) = {values[best_action]:.6f}")
    #print(f"  Выбранное оптимальное действие: {best_action} ({action_names[best_action]})")

# 6. Выводим итоговые результаты
# 6. Выводим итоговые результаты
print("\n5. ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("-"*60)

print(f"\nОптимальное среднее вознаграждение g* = {optimal_reward:.4f}")
print("\nОптимальная стационарная детерминированная политика:")
for state in range(num_states):
    print(f"  Состояние {state} ({state_names[state]}) → Действие {optimal_policy[state]} ({action_names[optimal_policy[state]]})")

print("\nСтационарное распределение по состояниям:")
for state in range(num_states):
    state_sum = sum(optimal_x[state * num_actions + action] for action in range(num_actions))
    print(f"  Состояние {state} ({state_names[state]}): {state_sum:.6f}")

# Дополнительная информация о базисных и небазисных переменных
print("\nИнформация о базисных и небазисных переменных:")
print("-"*60)

# Определяем порог для идентификации базисных переменных
threshold = 1e-10

# Находим базисные и небазисные переменные
basic_vars = []
nonbasic_vars = []

for i in range(num_variables):
    state = i // num_actions
    action = i % num_actions
    value = optimal_x[i]
    
    if abs(value) > threshold:
        basic_vars.append((i, state, action, value))
    else:
        nonbasic_vars.append((i, state, action, value))

# Выводим базисные переменные
print("\nБазисные переменные (положительные значения):")
for idx, state, action, value in basic_vars:
    print(f"  x({state},{action}) = {value:.6f} ({state_names[state]}, {action_names[action]})")

# Выводим небазисные переменные
print("\nНебазисные переменные (нулевые значения):")
for idx, state, action, value in nonbasic_vars:
    print(f"  x({state},{action}) = {value:.6e} ({state_names[state]}, {action_names[action]})")




Итоговый вектор правых частей b_eq:
  b_eq[0] (Уравнение баланса для состояния 0 (Отличный)) = 0.0
  b_eq[1] (Уравнение баланса для состояния 1 (Хороший)) = 0.0
  b_eq[2] (Уравнение баланса для состояния 2 (Удовлетворительный)) = 0.0
  b_eq[3] (Условие нормировки) = 1.0

Система уравнений в развернутом виде:
  0.7000·y(0,0) +0.8000·y(0,1) +0.7000·y(0,2) -0.2000·y(1,0) -0.1000·y(1,1) -0.2000·y(1,2) -0.1000·y(2,0) -0.1000·y(2,1) -0.1000·y(2,2) = 0.0
  -0.5000·y(0,0) -0.7000·y(0,1) -0.4000·y(0,2) +0.4000·y(1,0) +0.6000·y(1,1) +0.4000·y(1,2) -0.2000·y(2,0) -0.2000·y(2,1) -0.3000·y(2,2) = 0.0
  -0.2000·y(0,0) -0.1000·y(0,1) -0.3000·y(0,2) -0.2000·y(1,0) -0.5000·y(1,1) -0.2000·y(1,2) +0.3000·y(2,0) +0.3000·y(2,1) +0.4000·y(2,2) = 0.0

3. РЕШЕНИЕ ЗАДАЧИ ЛИНЕЙНОГО ПРОГРАММИРОВАНИЯ
------------------------------------------------------------

Задача: максимизировать r^T·x при ограничениях A_eq·x = b_eq, x ≥ 0
  где x - вектор переменных x(s,a), r - вектор моментальных вознаграждений
  При этом